# Physics 5300 Final Problem Set - Orbits with Leapfrog

Created 4-25-19 by Lucas Nestor  
Revised 4-25-19 by Lucas Nestor

In this notebook we will implement solving orbital equations using the leapfrog method.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
class Orbit():
    def __init__(self, m1=1, m2=1, G=1):
        self.m1 = m1
        self.m2 = m2
        self.G = G
    
    def r12(self, y):
        """
        Returns the relative distance between two particles.
        
        Parameters
        ==========
        y : float
            a vector with y[0] = x1, y[1] = y1, y[2] = x2, y[3] = y2
        """
        x1, y1, x2, y2 = y[0:4]
        return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    
    def dy_dt(self, t, y):
        """
        Returns the RHS of the d^2x/dt^2 and d^2y/dt^2 for masses 1 and 2.
        
        Parameters
        ==========
        t : float
            time
        y : float
            a vector with y[0] = x1, y[1] = y1, y[2] = x2, y[3] = y2
            y[4] = dx1/dt, y[5] = dy1/dt, y[6] = dx2/dt, and y[7] = dy2/dt
        """
        x1, y1, x2, y2 = y[0:4]
        
        dy_dt_vec = np.zeros(8)
        dy_dt_vec[0:4] = y[4:8]
        dy_dt_vec[4] = self.G * self.m2 * (x2 - x1) / self.r12(y)**3
        dy_dt_vec[5] = self.G * self.m2 * (y2 - y1) / self.r12(y)**3
        dy_dt_vec[6] = self.G * self.m1 * (x1 - x2) / self.r12(y)**3
        dy_dt_vec[7] = self.G * self.m1 * (y1 - y2) / self.r12(y)**3
        
        return dy_dt_vec
    
    def solve_ode(self, t_pts, initial_conditions, abserr=1.0e-10, relerr=1.0e-10):
        """
        Solves the motion for a given set of initial conditions.
        
        Parameters
        ==========
        t_pts : float
                time
        initial_conditions : float
                             array in the form of [x1, y1, x2, y2, x1_dot, y1_dot, x2_dot, y2_dot]
        abserr : float
                 absolute error threshold
        relerr : float
                relative error threshold
        """
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), initial_conditions,
                         t_eval=t_pts, atol=abserr, rtol=relerr).y
        
        return (solution[0:4], solution[4:8])
        
    def solve_ode_leapfrog(self, t_pts, initial_conditions):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        
        Parameters
        ==========
        t_pts : float
                time
        initial_conditions : float
                             array in the form of [x1, y1, x2, y2, x1_dot, y1_dot, x2_dot, y2_dot]

        """
        delta_t = t_pts[1] - t_pts[0]
        
        num_t_pts = len(t_pts)
        pos = np.array([np.zeros(num_t_pts), np.zeros(num_t_pts), np.zeros(num_t_pts), np.zeros(num_t_pts)]).T
        vel = np.array([np.zeros(num_t_pts), np.zeros(num_t_pts), np.zeros(num_t_pts), np.zeros(num_t_pts)]).T
        vel_half = np.array([np.zeros(num_t_pts), np.zeros(num_t_pts), np.zeros(num_t_pts), np.zeros(num_t_pts)]).T
        
        # initial conditions
        pos[0] = initial_conditions[0:4]
        vel[0] = initial_conditions[4:8]
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            y = flatten([pos[i], vel[i]])
            vel_half[i] = vel[i] + self.dy_dt(t, y)[4:8] * delta_t / 2.
            pos[i+1] = pos[i] + vel_half[i] * delta_t
            
            y = flatten([pos[i+1], vel[i]])
            vel[i+1] = vel_half[i] + self.dy_dt(t, y)[4:8] * delta_t / 2.
        
        return pos.T, vel.T                                               

## Comparing Methods

Now we will look at a variety of orbits and compare `solve_ivp` to the leapfrog method.

In [ ]:
def plot_orbits(pos_lf, pos_ivp):
    fig = plt.figure(figsize=(12,4))

    ax1 = fig.add_subplot(1,2,1)
    ax1.plot(pos_lf[0], pos_lf[1])
    ax1.plot(pos_lf[2], pos_lf[3])
    ax1.set_title('Leapfrog')

    ax2 = fig.add_subplot(1,2,2)
    ax2.plot(pos_ivp[0], pos_ivp[1])
    ax2.plot(pos_ivp[2], pos_ivp[3])
    ax2.set_title('SciPy solve_ivp')

In [ ]:
m1 = m2 = 1.

o = Orbit()
t_pts = np.arange(0., 150., 0.1)

x1_0 = 5
x2_0 = -5

y1_0 = y2_0 = 0
x1_dot_0 = x2_dot_0 = 0

y1_dot_0 = .2
y2_dot_0 = -m1 / m2 * y1_dot_0

initial_conditions = [x1_0, y1_0, x2_0, y2_0, x1_dot_0, y1_dot_0, x2_dot_0, y2_dot_0]

pos_lf, _ = o.solve_ode_leapfrog(t_pts, initial_conditions)
pos_ivp, _ = o.solve_ode(t_pts, initial_conditions)

plot_orbits(pos_lf, pos_ivp)

In [ ]:
m1 = 2.
m2 = 1.

ratio = -m1 / m2

x1_0 = 5
x2_0 = -5
y1_0 = y2_0 = x1_dot_0 = x2_dot_0 = 0

y1_dot_0 = .1
y2_dot_0 = ratio * y1_dot_0

initial_conditions = [x1_0, y1_0, x2_0, y2_0, x1_dot_0, y1_dot_0, x2_dot_0, y2_dot_0]

o = Orbit(m1=m1, m2=m2)
t_pts = np.arange(0, 60., 0.01)

pos_lf, _ = o.solve_ode_leapfrog(t_pts, initial_conditions)
pos_ivp, _ = o.solve_ode(t_pts, initial_conditions)

plot_orbits(pos_lf, pos_ivp)

In [ ]:
m1 = 100.
m2 = 1.

ratio = -m1 / m2

x1_0 = -10
x2_0 = 10
y1_0 = y2_0 = x1_dot_0 = x2_dot_0 = 0

y1_dot_0 = .01
y2_dot_0 = ratio * y1_dot_0

initial_conditions = [x1_0, y1_0, x2_0, y2_0, x1_dot_0, y1_dot_0, x2_dot_0, y2_dot_0]

o = Orbit(m1=m1, m2=m2)

t_pts = np.arange(0, 50., 0.01)

pos_lf, _ = o.solve_ode_leapfrog(t_pts, initial_conditions)
pos_ivp, _ = o.solve_ode(t_pts, initial_conditions)

plot_orbits(pos_lf, pos_ivp)
